# STAT 201 Midterm #

*************************************************************
* **Author**:		Alexis Harris
* **Filename**:		STAT201_Midterm_AHarris.ipynb
* **Date Created**:	11/4/2020
* **Modifications**:	None
			
**************************************************************

*************************************************************

* Lab/Assignment: Stat 201 Midterm


 
* Overview: This file contains the answers to the Stat 201 Midterm due Nov. 4, 2020.


* Input: Input datasets from google.cloud's bigquery for problem 4. Also used Melbourne_housing_FULL.csv which should be uploaded to this notebook.


* Output: Queries from google.cloud's data and Dataframes from csv file


************************************************************

## **1.)  List and explain the steps in a typical data science project workflow.**

1. First, you should discuss with your boss or company what problem they want you to solve and what data you will need access to.
2. List the data issues to fix and make the data usable.
3. Produce a preliminary report on the data and figure out the questions that can be answered from the data description. May need to repeat the process again.
4. List questions of interest.
5. Attempt to thouroughly clean the data.
6. Load Dataset. If not loading correctly, you need to clean again.
7. Explore the data! Produce a descriptive analysis, visual data, communicate. See if the data creates more questions.
8. Move to machine learning (perform/learn some task) - test and train data. This needs to answer original questions.
9. Report findings.
10. Celebrate!

## **2.)  List at least three common problems that a data scientist might encounter when working with an employer and potential work-arounds for these problems.**
1. Miscommunication - sometimes you and your employer may not be on the same page for what problem you should be working on. Make sure to communicate effectively so you can answer your employers question.
2. Expecting a specific answer - I think sometimes the questions an employer asks a data scientist will not always have an answer that he or she wants to hear. The company might not have enough data for the data scientist to use to conclude a statistically sound answer. A workaround to this would be to try to find out near the beginning of the workflow whether the data the employer provides is usable or not.
3. The employer not using the results found from the data science project - Sometimes the employer or company might not utilize the results found from a data scientist in their decision making process. Data scientists might have to deal with defending their work to the company if that company doesn't prioitize data science.

## **3.)  What is the major difference between a database and a flat file and why are databases more useful in many situations?**

The major difference between a database and a flat file is that a database has multiple data structures. This means there are multiples tables of information in a database. On the other hand, a flat file will only have one file with all the data information stored in one place. Databases are more useful because they are easier to navigate compared to a flat file and there is less chance of data redunancies/errors to occur.

## **4.)  For this question, we will be using SQL and the Bigquery environment.  Please provide code that is structured and commented according to the CST style guide posted on the course webpage.**

**a.) Within Kaggle is a database of SanFransisco public data called “san_francisco”  Start by reading this data.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from google.cloud import bigquery

In [ ]:
client = bigquery.Client()  # import Kaggle's bigquery data
dataset_ref = client.dataset("san_francisco",project = "bigquery-public-data") # Call san_francisco dataset from bigquery data
sfDataset = client.get_dataset(dataset_ref)

**b.)  Produce a list of the tables within this dataset.**

In [ ]:
tables = list(client.list_tables(sfDataset)) # create a list to view the tables within the san_francisco dataset
for table in tables:
    print(table.table_id)

**c.)  Determine the size of data required to import the film_locations table without actually loading it.**

In [ ]:
# create a query of the film_locations dataset. Will select everything from this table and see how many bytes this query will process
query1 = """
    SELECT *
    FROM bigquery-public-data.san_francisco.film_locations
"""
dry_run_config = bigquery.QueryJobConfig(dry_run = True) #dry run
dry_run_query_job = client.query(query1, job_config = dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed)) #line of code that views total bytes processed

**d.)  Read this table and produce a data frame that counts the number of movies produced by each production company from before 1950 and listed from most movies to least.**

In [ ]:
table_ref_film = dataset_ref.table('film_locations') # look at film_locations table
filmLocations = client.get_table(table_ref_film)
client.list_rows(filmLocations, max_results = 6).to_dataframe() # look at 6 rows of the film_locations data

In [ ]:
OneHundMB = 1000*1000*100 # create a bytes size number that will process a query less than or equal to 100MB
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = OneHundMB) 
#set a safe congifuration number so that we don't go over 100MB

In [ ]:
# Create a query with a list of movies from each production company before 1950, from highest number to lowest amount of movies.
query2 = """
    SELECT 
        COUNT(title) AS Movie_Count,  
        production_company,
    FROM `bigquery-public-data.san_francisco.film_locations`
    WHERE 
        release_year < 1950
    GROUP BY 
        production_company
    ORDER BY
        Movie_Count DESC

"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = OneHundMB) #need to re-run safe_config line everytime you creat a new query
filmLocations = client.query(query2, job_config=safe_config) #set safe config
filmLocations.to_dataframe() #output

**e.)  From the table sfpd_incidents, produce a table of the most common police incidents in the category of LARCENY/THEFT in the year 2016 ordered from most to least common.**

In [ ]:
table_ref_sfpd_incidents = dataset_ref.table('sfpd_incidents') # look at sfpd_incidents table
sfpdIncidents = client.get_table(table_ref_sfpd_incidents)
client.list_rows(sfpdIncidents, max_results = 6).to_dataframe() # look at 6 rows of the sfpd_incidents data

In [ ]:
# This query will produce a table of the most common police incidents in the category of LARCENY/THEFT 
# in the year 2016 ordered from most to least common.
query3 = """
    SELECT
        descript,
        COUNT(descript) AS Num_Incidences
    FROM `bigquery-public-data.san_francisco.sfpd_incidents`
    WHERE 
        EXTRACT(YEAR FROM timestamp) = 2016
        AND
        category = 'LARCENY/THEFT'
    GROUP BY 
        descript
    ORDER BY 
        COUNT(descript) DESC

"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = OneHundMB*10)
sfpdIncidents = client.query(query3, job_config=safe_config)
sfpdIncidents.to_dataframe()

## **5.)  For this question, I would like you to use Pandas and Python to retrieve and clean a dataset according the following instructions.  As usual, please provide code that is structured and commented according to the CST style guide posted on the course webpage.**

**a.) Using the Kaggle dataset on Melborne_housing_FULL.csv, start by reading and describing this dataset.**

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../input/melbourne-houses/Melbourne_housing_FULL.csv")#read in dataset as df

In [ ]:
df.shape #look at shape of df
df.describe() #overview of content in df

**b.)  Using the iloc method, produce the last 10 rows of the dataset.**

In [ ]:
df.iloc[-10:] #select last ten rows

**c.)  There are columns with missing data in this dataset.  Clean the dataset by removing all rows with missing values.  (This is a BAD idea, but it is where we start).**

In [ ]:
df = df.dropna() #drops rows with missng values
df.head()

**d.)  Using python code, produce rows where the Suburb is in Albert Park and the house has 3 or more bedrooms and sold in 2017.  Print the 5 observations with lowest price where the price is not null.**

In [ ]:
#need to split up the 'Date' column since it is not timestamped
df.Date.dtype

In [ ]:
#Add three new columns Month, Day, and Year to the end of the dataset
df[['Month','Day','Year']] = df['Date'].str.split('/',expand=True)
df.head()

In [ ]:
#create a new df with the problem's conditions
sub_Albert = df.loc[(df.Suburb == 'Albert Park') & (df.Rooms >= 3) & (df.Year == '2017')] 
sub_Albert.head()

**e.)  Produce a list of all the unique Sellers.**

In [ ]:
df.SellerG.unique() #show unique sellers